In [14]:
import os
import pandas as pd
import sqlite3 as sqlite

In [15]:
data = pd.read_csv("scraped.csv").query("team != 'team/'")

In [16]:
names = pd.read_csv("names.csv", encoding="utf-8")

In [17]:
sport_ents = (
    names
    .reset_index()
    .to_numpy()
    .tolist()
)
id_mapping = {f"{'team' if year else 'rider'}/{url}":i for i, _, url, year in sport_ents}

In [18]:
graph = pd.DataFrame()
graph["A"] = data["rider"].map(id_mapping)
graph["B"] = data["team"].map(id_mapping)

graph_sql = graph.to_numpy().tolist() + graph.to_numpy()[:,::-1].tolist()

In [19]:
path_to_db = "../docs/graph.sqlite"
try:
    try:
        os.remove(path_to_db)
    except:
        conn.close()
        del conn
        os.remove(path_to_db)
except FileNotFoundError:
    pass

In [20]:
with sqlite.connect(path_to_db) as conn:
    conn.execute("""CREATE TABLE sport_ents (
                    id int PRIMARY KEY,
                    name text NOT NULL,
                    url text NOT NULL,
                    year int NOT NULL
                ) WITHOUT ROWID""")
    conn.executemany("INSERT INTO sport_ents VALUES (?, ?, ?, ?)", sport_ents)
    conn.execute("""CREATE TABLE graph (
                    a int NOT NULL,
                    b int NOT NULL)""")
    conn.executemany("INSERT INTO graph VALUES (?, ?)", graph_sql)
    conn.execute("CREATE INDEX graph_roots ON graph (a) ")
conn.close()
del conn